### 案例

*例子来源于李航的《统计学习方法》*
假设我们有3个盒子，每个盒子里都有红色和白色两种球，这三个盒子里球的数量分别是：


|||
:--:|--|--|--
盒子|	1|	2|	3
红球数	|5	|4	|7
白球数	|5|	6	|3

按照下面的方法从盒子里抽球:
开始的时候，从第一个盒子抽球的概率是0.2，从第二个盒子抽球的概率是0.4，从第三个盒子抽球的概率是0.4。
以这个概率抽一次球后，将球放回。然后从当前盒子转移到下一个盒子进行抽球。
`规则是`：
- 如果当前抽球的盒子是第一个盒子，则以0.5的概率仍然留在第一个盒子继续抽球，以0.2的概率去第二个盒子抽球，以0.3的概率去第三个盒子抽球。
- 如果当前抽球的盒子是第二个盒子，则以0.5的概率仍然留在第二个盒子继续抽球，以0.3的概率去第一个盒子抽球，以0.2的概率去第三个盒子抽球。
- 如果当前抽球的盒子是第三个盒子，则以0.5的概率仍然留在第三个盒子继续抽球，以0.2的概率去第一个盒子抽球，以0.3的概率去第二个盒子抽球。

如此下去，直到重复三次，得到一个球的颜色的观测序列:

$O=\begin{Bmatrix} {\text{红，白，红}} \end{Bmatrix}$

`注意在这个过程中，观察者只能看到球的颜色序列，却不能看到球是从哪个盒子里取出的。`

观察集合是:
$V=\begin{Bmatrix} {\text{红，白}} \end{Bmatrix},M=2$

状态集合是：
$Q=\begin{Bmatrix} {\text{盒子1，盒子2，盒子3}} \end{Bmatrix}$

初始状态分布为：
$\Pi=(0.2,0.4,0.4)^T$

状态转移概率分布矩阵为：

$A=
        \begin{pmatrix}
        0.5 & 0.2 & 0.3 \\
        0.3 & 0.5 & 0.2 \\
        0.2 & 0.3 & 0.5 \\
        \end{pmatrix}
$

观测状态概率矩阵为：

$B=
        \begin{pmatrix}
        0.5 & 0.5 \\
        0.4 & 0.6 \\
        0.7 & 0.3 \\
        \end{pmatrix}
$

### 代码

In [1]:
#定义数据
Hidden_states = ("box 1", "box 2", "box 3") # 隐状态集合
Observations_states = ('red', 'white', 'red') # 观测状态集合

Start_probability = {'box 1': 0.2, 'box 2': 0.4, 'box 3': 0.4} # 初始状态

Hidden_transition_probability = { # 隐马尔可夫链中从当前盒子转移到其他盒子的概率
    'box 1' : {'box 1': 0.5, 'box 2': 0.2, 'box 3': 0.3},
    'box 2' : {'box 1': 0.3, 'box 2': 0.5, 'box 3': 0.2},
    'box 3' : {'box 1': 0.2, 'box 2': 0.3, 'box 3': 0.5},
   }

Hidden_observations_probability = {    # 原来叫emission_probability。这里表示每个盒子里面红白球的数量
    'box 1' : {'red': 0.5, 'white': 0.5},
    'box 2' : {'red': 0.4, 'white': 0.6},
    'box 3' : {'red': 0.7, 'white': 0.3},
   }

In [2]:
def print_dptable(V): # 打印矩阵
    print ("    "),
    for i in range(len(V)): 
        print("%7d" % i)
    print()

    for y in V[0].keys():
        print ("%.5s: " % y)
        for t in range(len(V)):
            print ("%.7s" % ("%f" % V[t][y]))
        print()

In [3]:
def viterbi(obs, states, start_p, trans_p, h2o_p): # Viterbi算法
    '''
    input:
        obs:观测状态集合O={红，白，红}
        states：隐状态集合Q={盒子1，盒子2，盒子3}
        start_p：初始状态Π=(0.2,0.4,0.4)T
        trans_p：转移状态矩阵A=【0.5 0.3 0.2
                              0.2 0.5 0.3
                              0.3 0.2 0.5】
        h2o_p：观测状态概率矩阵为：B=【0.5 0.4 
                                   0.7 0.5 
                                   0.6 0.3】
    output:
        prob:最优概率
        path[state]：最优路径
    '''
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        #t=1时刻，状态为i观测o1为红的概率，即δ1(i)
        V[0][y] = start_p[y] * h2o_p[y][obs[0]]
        # 初始状态，由start的概率，对应乘上发射概率，即由隐状态到观测状态的可能性
        path[y] = [y]

    # 开始遍历，t=2和t=3时刻
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}

        for y in states:
            # 对于每个箱子，计算其由前一个的各个状态，到现在箱子的概率大小，取最大值。即求出最有可能到达现在箱子的路径
            # 前一个箱子转移到现在箱子的每个状态对应的路径大小都计算了，取最大值。作为V[t][y]的值，并更新路径
            (prob, state) = max([(V[t-1][y0] * trans_p[y0][y] * h2o_p[y][obs[t]], y0) for y0 in states])
            V[t][y] = prob
            newpath[y] = path[state] + [y]

        # Don't need to remember the old paths
        path = newpath

    print_dptable(V)
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])
    return (prob, path[state])

In [4]:
viterbi(Observations_states,
                   Hidden_states,
                   Start_probability,
                   Hidden_transition_probability,
                   Hidden_observations_probability)

    
      0
      1
      2

box 1: 
0.10000
0.02800
0.00756

box 2: 
0.16000
0.05040
0.01008

box 3: 
0.28000
0.04200
0.01470



(0.014699999999999998, ['box 3', 'box 3', 'box 3'])

### hmmlearn包计算

In [5]:
import numpy  as np
from hmmlearn import hmm

In [6]:


states = ["box 1", "box 2", "box3"]
n_states = len(states)

observations = ["red", "white"]
n_observations = len(observations)

start_probability = np.array([0.2, 0.4, 0.4])

transition_probability = np.array([
  [0.5, 0.2, 0.3],
  [0.3, 0.5, 0.2],
  [0.2, 0.3, 0.5]
])

emission_probability = np.array([
  [0.5, 0.5],
  [0.4, 0.6],
  [0.7, 0.3]
])


In [7]:

model = hmm.MultinomialHMM(n_components=n_states)
model.startprob_=start_probability
model.transmat_=transition_probability
model.emissionprob_=emission_probability

In [8]:
seen = np.array([[0,1,0]]).T
logprob, box = model.decode(seen, algorithm="viterbi")

In [9]:
print("The ball picked:【", ", ".join(map(lambda x: observations[x[0]], seen)),"】")


The ball picked:【 red, white, red 】


In [10]:
print("The hidden box【", ", ".join(map(lambda x: states[x], box)),"】")

The hidden box【 box3, box3, box3 】


In [11]:
logprob

-4.219907785197447

In [12]:
np.exp(logprob)

0.014699999999999996